# Notes


```
augmentation: True
	backbone: resnet
	batch_size: 64
	class_weights: True
	data_path: data/4_tfds_dataset/
	dataset_id: [1]
	dropout_rate: 0.3
	earlystopping_patience: 20
	epochs: 100
	image_channels: 3
	image_size: 224
	loss: categorical_crossentropy
	lr: 0.0001
	metrics: ['accuracy']
	monitor: val_accuracy
	notes: 
	num_classes: 7
	optimizer: adam
	preprocess: True
	project: Whats-this-rockv3
	reduce_lr_factor: 0.5
	reduce_lr_min_lr: 1e-05
	reduce_lr_patience: 7
	sampling: undersampling
	save_model: False
	seed: 42
	trainable: False
	use_earlystopping: True
	use_pretrained_weights: True
	use_reduce_lr: True
	use_wandb: True
	wand_mode: online
```


- remove corrupted images - Takes 18 secs
- Try undersampling with this, and all models
        python src/models/train.py 'notes="finetuning, mode:max"' \
                            wandb.project=Whats-this-rockv3 \
                            dataset.id=[1] \
                            dataset.sampling=oversampling \
                            epochs=50 \
                            lr=1e-2 \
                            optimizer=adamax \
                            model.backbone=mobilenetv2 \
                            model.dropout_rate=0.2 \
                            model.trainable=False \
                            callback.reduce_lr.use=False \
                            callback.earlystopping.patience=20 \
                            model.save_model=False
- improve using these
    - [Finetuning Efficientnet](https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/vision/ipynb/image_classification_efficientnet_fine_tuning.ipynb#scrollTo=1KvkMvLI0iCh)
    - [Wandb Article](https://wandb.ai/stacey/mendeleev/reports/Tables-Tutorial-Visualize-Data-for-Image-Classification--VmlldzozNjE3NjA)
    - [bloodMNIST](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/keras/Keras_pipeline_with_Weights_and_Biases.ipynb#scrollTo=-303Shsata7Z)

    - [Simpsons classification](https://colab.research.google.com/drive/181GCGp36_75C2zm7WLxr9U2QjMXXoibt#scrollTo=xftEwKyuaJ5q)

## Efficientnet Finetuning Notes

- too low- [0.000001, 0.000005, 0.00005]
- too high [0.002, 0.005]
- 0.001, 0.0005 works
    - why?
    - isn't it too high for finetuning?

## Efficientnet to do
- should lr be min max or a list [0.01, 0.005, 0.001, 0.0005, 0.0001]
- freeze parameter - [true. false]
    - so 2 best models, one with freeze true and another with freeze false
    - [freeze: true best model](https://wandb.ai/rock-classifiers/Whats-this-rockv2/runs/9fgg9eua?workspace=user-udaylunawat)
        
    val_accuracy - 63.9

    val_f1_score - 63.4
            earlystopping_patience 3
            freeze true
            lr 0.00007
            lr_reduce_factor 0.9
            lr_reduce_patience 1
    - [freeze: false best model](https://wandb.ai/rock-classifiers/Whats-this-rockv2/runs/4qmss0xh/overview?workspace=user-udaylunawat)
    
    val_accuracy:-0.74
    
    val_f1_score:-0.73

            lr:- 0.0003
            reduce_factor: 0.3
            lr_reduce_patience:- 3
            earlystopping_patience:- 10

    - finetune both models in another sweep

# Clone Repo

In [ ]:
!nvidia-smi

Sun Sep 18 12:04:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd /content/
!rm -rf /content/Whats-this-rock/
# !git clone https://github.com/udaylunawat/Whats-this-rock.git
!git clone -b hydra https://github.com/udaylunawat/Whats-this-rock.git
%cd /content/Whats-this-rock/

[Errno 2] No such file or directory: '/content/Whats-this-rock/'
/content
Cloning into 'Whats-this-rock'...
remote: Enumerating objects: 2241, done.
remote: Counting objects: 100% (481/481), done.
remote: Compressing objects: 100% (236/236), done.
remote: Total 2241 (delta 242), reused 411 (delta 242), pack-reused 1760
Receiving objects: 100% (2241/2241), 5.85 MiB | 19.64 MiB/s, done.
Resolving deltas: 100% (1445/1445), done.
/content/Whats-this-rock


# Training and evaluating Model

# Train

In [ ]:
import os
os.environ['WANDB_MODE'] = 'online' # offline

if 'WANDB_API_KEY' not in os.environ:
    if os.environ['WANDB_MODE'] == 'online':
        from getpass import getpass
        secret = getpass('Enter WandB API Key: ')
        os.environ['WANDB_API_KEY'] = secret
    else:
        print("WandB Offline!")

Enter WandB API Key: ··········


In [ ]:
!sh src/scripts/setup.sh

The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 18 not upgraded.
Need to get 430 MB of archives.
After this operation, 3,139 MB disk space will be freed.
(Reading database ... 155569 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 155547 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.0.77-1+cuda11.2) over (8.0.5.39-1+cuda11.1) ...
Setting up libcudnn8 (8.1.0.77-1+cuda11.2) ...
     |████████████████████████████████| 1.8 MB 35.5 MB/s 
     |████████████████████████████████| 151 kB 70.2 MB/s 
     |████████████████████████████████| 158 kB 

In [ ]:
!git pull

Already up to date.


In [ ]:
!python src/models/train.py project=Whats-this-rockv3 \
                            wandb.mode=offline \
                            dataset_id=[1,2,3,4] \
                            sampling=oversampling \
                            epochs=50 \
                            lr=0.0005 \
                            optimizer=adamax \
                            backbone=resnet \
                            dropout_rate=0.3 \
                            trainable=False \
                            use_reduce_lr=True \
                            reduce_lr_factor=0.7 \
                            reduce_lr_patience=3 \
                            reduce_lr_min_lr=0.00005 \
                            earlystopping_patience=20 \
                            save_model=False \
                            # 'notes="No notes."' \

wandb: Tracking run with wandb version 0.13.3
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
wandb: Adding directory to artifact (./src)... Done. 0.1s

Datasets used for Training:- [1, 2, 3, 4]
 92% 146M/159M [00:05<00:00, 31.6MB/s]
100% 159M/159M [00:05<00:00, 31.4MB/s]
renamed 'data/1_extracted/Dataset' -> 'data/1_extracted/dataset1'
 96% 337M/349M [00:11<00:00, 34.1MB/s]
100% 349M/349M [00:11<00:00, 32.4MB/s]
 98% 585M/595M [00:29<00:00, 24.5MB/s]
100% 595M/595M [00:29<00:00, 21.2MB/s]

Processing dataset3
Moving Granite: 100% 45/45 [00:00<00:00, 10244.45it/s]
Moving Limestone: 100% 57/57 [00:00<00:00, 11296.85it/s]
Moving Quartzite: 100% 64/64 [00:00<00:00, 12052.60it/s]
Moving Marble: 100% 70/70 [00:00<00:00, 12021.51it/s]
Moving Basalt: 100% 39/39 [00:00<00:00, 11027.23it/s]
Moving Sandstone: 100% 47/47 [00:00<00:00, 11814.94it/s]
Moving Granite: 100% 20/20 [00:00<00:00, 9873.60it/s]
Moving L

In [ ]:
# !python src/evaluate.py

# Deploy Telegram Bot
Uploading secrets with telegram key

In [ ]:
import os

from getpass import getpass
telegram_token = getpass('Enter Telegram API Key: ')
os.environ['TOKEN'] = telegram_token

In [ ]:
%%bash
pip install -r requirements.txt --quiet
python src/bot.py

# Weights and Biases Sweeps

In [ ]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/udaylunawat/Whats-this-rock
   1c1e59a..c6beb4d  hydra      -> origin/hydra
Updating 1c1e59a..c6beb4d
Fast-forward
 configs/sweep.yaml | 7 ++++++-
 1 file changed, 6 insertions(+), 1 deletion(-)


In [ ]:
!wandb sweep \
    --project Whats-this-rockv3 \
    --entity rock-classifiers \
    configs/sweep.yaml

wandb: Creating sweep from: configs/sweep.yaml
wandb: Created sweep with ID: qb1s1zx7
wandb: View sweep at: https://wandb.ai/rock-classifiers/Whats-this-rockv3/sweeps/qb1s1zx7
wandb: Run sweep agent with: wandb agent rock-classifiers/Whats-this-rockv3/qb1s1zx7


In [ ]:
!wandb agent rock-classifiers/Whats-this-rockv3/qb1s1zx7

wandb: Starting wandb agent 🕵️
2022-09-17 00:00:36,552 - wandb.wandb_agent - INFO - Running runs: []
2022-09-17 00:00:36,700 - wandb.wandb_agent - INFO - Agent received command: run
2022-09-17 00:00:36,700 - wandb.wandb_agent - INFO - Agent starting run with config:
	augmentation: True
	backbone: mobilenetv2
	batch_size: 64
	class_weights: True
	data_path: data/4_tfds_dataset/
	dataset_id: [1]
	dropout_rate: 0.3
	earlystopping_patience: 20
	epochs: 100
	image_channels: 3
	image_size: 224
	loss: categorical_crossentropy
	lr: 1e-05
	metrics: ['accuracy']
	monitor: val_accuracy
	notes: 
	num_classes: 7
	optimizer: adamax
	preprocess: True
	project: Whats-this-rockv3
	reduce_lr_factor: 0.3
	reduce_lr_min_lr: 1e-05
	reduce_lr_patience: 7
	sampling: None
	save_model: False
	seed: 42
	trainable: True
	use_earlystopping: True
	use_pretrained_weights: True
	use_reduce_lr: True
	use_wandb: True
	wand_mode: online
2022-09-17 00:00:36,705 - wandb.wandb_agent - INFO - About to run command: /usr/bin